In [1]:

import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage

In [2]:
class Layer:
    # GRADED FUNCTION: initialize_parameters

    def initialize_parameters(n_x, n_h, n_y):
        """
        Argument:
        n_x -- size of the input layer
        n_h -- size of the hidden layer
        n_y -- size of the output layer

        Returns:
        parameters -- python dictionary containing your parameters:
                        W1 -- weight matrix of shape (n_h, n_x)
                        b1 -- bias vector of shape (n_h, 1)
                        W2 -- weight matrix of shape (n_y, n_h)
                        b2 -- bias vector of shape (n_y, 1)
        """

        np.random.seed(1)

        ### START CODE HERE ### (≈ 4 lines of code)
        W1 = np.random.randn(n_h, n_x)*0.01
        b1 = np.zeros((n_h,1))
        W2 = np.random.randn(n_y, n_h)*0.01
        b2 = np.zeros((n_y,1))
        ### END CODE HERE ###

        assert(W1.shape == (n_h, n_x))
        assert(b1.shape == (n_h, 1))
        assert(W2.shape == (n_y, n_h))
        assert(b2.shape == (n_y, 1))

        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2}

        return parameters  
    
    # GRADED FUNCTION: initialize_parameters_deep

    def initialize_parameters_deep(layer_dims):
        """
        Arguments:
        layer_dims -- python array (list) containing the dimensions of each layer in our network

        Returns:
        parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                        Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                        bl -- bias vector of shape (layer_dims[l], 1)
        """

        np.random.seed(3)
        parameters = {}
        L = len(layer_dims)            # number of layers in the network

        for l in range(1, L):
            ### START CODE HERE ### (≈ 2 lines of code)
            parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
            parameters['b' + str(l)] =  np.zeros((layer_dims[l], 1))
            ### END CODE HERE ###

            assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
            assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))


        return parameters
    #linear forward
    def linear_forward(A, W, b):
        """
        Implement the linear part of a layer's forward propagation.

        Arguments:
        A -- activations from previous layer (or input data): (size of previous layer, number of examples)
        W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
        b -- bias vector, numpy array of shape (size of the current layer, 1)

        Returns:
        Z -- the input of the activation function, also called pre-activation parameter 
        cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
        """

        Z = np.dot(W, A) + b

        assert(Z.shape == (W.shape[0], A.shape[1]))
        cache = (A, W, b)

        return Z, cache
    
    def linear_activation_forward(A_prev, W, b, activation):
 
        # cache is stored to be used in backward propagation module
        if activation == "sigmoid":
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = sigmoid(Z)
        elif activation == "Hyperbolic_Tan":
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = tanh(Z)
            
        elif activation == "Softmax":
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = softmax(Z)
            
        '''elif activation == "Cross_Entropy_Loss":
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = cross_entropy_loss(Z,)
            

            
        
        cache = (linear_cache, activation_cache)

        return A, cache
    def cross_entropy_loss(Z, target):
        return -target * np.log(Z)'''
    def sigmoid(Z):
        
        A = 1/(1 + np.exp(-Z))
        return A, {'Z' : Z}

    def tanh(Z):

        A = np.tanh(Z)
        return A, {'Z' : Z}
    def softmax(Z):
        num = np.exp(Z - np.max(Z))

        return num / np.sum(num, axis=0, keepdims=True), {'Z' : Z}
    
    
    def L_model_forward(X, parameters,activation_func):
        """
        Arguments:
        X -- data, numpy array of shape (input size, number of examples)
        parameters -- output of initialize_parameters_deep()

        Returns:
        AL -- last post-activation value
        caches -- list of caches containing:
                    every cache of linear_activation_forward()
               (there are L-1 of them, indexed from 0 to L-1)
        """

        caches = []
        A = X

        # number of layers in the neural network
        L = len(parameters) // 2                 

        # Implement [LINEAR -> TANH]*(L-1). Add "cache" to the "caches" list.
        for l in range(L):
            A_prev = A
            A, cache = linear_activation_forward(A_prev,
                               parameters['W' + str(l)],
                       parameters['b' + str(l)], activation_func)

            caches.append(cache)

        # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
        #AL, cache = linear_activation_forward(A, parameters['W' + str(L)],parameters['b' + str(L)], 'sigmoid')
        caches.append(cache)

        return AL, caches
    
    def compute_cost(AL, Y):
    
        #Implement the cost function defined by the equation.
        m = Y.shape[1]
        cost = (-1 / m)*(np.dot(np.log(AL), Y.T)+np.dot(np.log((1-AL)), (1 - Y).T))

        # To make sure your cost's shape is what we
        # expect (e.g. this turns [[20]] into 20).
        cost = np.squeeze(cost)     

        return cost
    
    def linear_activation_backward(dA, cache, activation):
        """
        Implement the backward propagation for the LINEAR->ACTIVATION layer.

        Arguments:
        dA -- post-activation gradient for current layer l 
        cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
        activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

        Returns:
        dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
        dW -- Gradient of the cost with respect to W (current layer l), same shape as W
        db -- Gradient of the cost with respect to b (current layer l), same shape as b
        """
        linear_cache, activation_cache = cache

        if activation == "relu":
            ### START CODE HERE ### (≈ 2 lines of code)
            dZ = relu_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward(dZ, linear_cache)
            ### END CODE HERE ###

        elif activation == "sigmoid":
            ### START CODE HERE ### (≈ 2 lines of code)
            dZ = sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward(dZ, linear_cache)
            ### END CODE HERE ###
            
        elif activation == "Hyperbolic_tan":
            ### START CODE HERE ### (≈ 2 lines of code)
            dZ = tanh_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward(dZ, linear_cache)
            ### END CODE HERE ###
            
        elif activation == "softmax":
            ### START CODE HERE ### (≈ 2 lines of code)
            dZ = softmax_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward(dZ, linear_cache)
            ### END CODE HERE ###

        return dA_prev, dW, db
    
    def softmax_backward(dA, activation_cache):
        
        Z = activation_cache['Z']
        
        return (np.einsum('ij,jk->ijk', Z, np.eye(Z.shape[-1])) - np.einsum('ij,ik->ijk', Z, Z))
        
    
    def linear_backward(dZ, cache):

        A_prev, W, b = cache
        m = A_prev.shape[1]
        dW = (1 / m)*np.dot(dZ, A_prev.T)
        db = (1 / m)*np.sum(dZ, axis = 1, keepdims = True)
        dA_prev = np.dot(W.T, dZ)

        return dA_prev, dW, db
    
    
    def sigmoid_backward(dA, activation_cache):

        Z = activation_cache['Z']
        A = sigmoid(Z)
        return dA * (A*(1 - A))    # A*(1 - A) is the derivative of sigmoid function

    def tanh_backward(dA, activation_cache):

        Z = activation_cache['Z']
        A = sigmoid(Z)
        return dA * (1 -np.power(A, 2)) 
    
    def L_model_backward(AL, Y, caches):
        """
        AL -- probability vector, output of the forward propagation (L_model_forward())
        Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
        caches -- list of caches containing:
                    every cache of linear_activation_forward() with "tanh"
                    (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                    the cache of linear_activation_forward() with "sigmoid"
                     (it's caches[L-1])

        Returns:
        grads -- A dictionary with the gradients
                 grads["dA" + str(l)] = ...
                 grads["dW" + str(l)] = ...
                 grads["db" + str(l)] = ...
        """
        grads = {}
        L = len(caches) # the number of layers
        m = AL.shape[1]
        Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

        # Initializing the backpropagation
        # derivative of cost with respect to AL

        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

        # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache".
        # Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
        current_cache = caches[L - 1]
        grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = \
                      linear_activation_backward(dAL, current_cache, 'sigmoid')

        # Loop from l = L-2 to l = 0
        for l in reversed(range(L-1)):
            current_cache = caches[l]
            dA_prev_temp, dW_temp, db_temp = linear_activation_backward(
                        grads['dA' + str(l + 1)], current_cache, 'tanh')
            grads["dA" + str(l)] = dA_prev_temp
            grads["dW" + str(l + 1)] = dW_temp
            grads["db" + str(l + 1)] = db_temp

        return grads
    def update_parameters(parameters, grads, learning_rate):
        L = len(parameters) // 2 # number of layers in the neural network

        # Update rule for each parameter. Use a for loop.
        for l in range(L):
            parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads['dW' + str(l + 1)]
            parameters["b" + str(l + 1)] = parameters['b' + str(l + 1)] - learning_rate * grads['db' + str(l + 1)]

        return parameters
    
    #training the model
    def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost = False):
        """
        Arguments:
        X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
        Y -- true "label" vector (containing 0 if cat, 1 if non-cat),
                                     of shape (1, number of examples)
        layers_dims -- list containing the input size and each layer size,
                                          of length (number of layers + 1).
        learning_rate -- learning rate of the gradient descent update rule
        num_iterations -- number of iterations of the optimization loop
        print_cost -- if True, it prints the cost every 100 steps

        Returns:
        parameters -- parameters learned by the model. They can then be used to predict.
        """

        np.random.seed(1)
        costs = []                         # keep track of cost

        parameters = initialize_parameters_deep(layers_dims)

        # Loop (gradient descent)
        for i in range(0, num_iterations):

            # Forward propagation: [LINEAR -> TANH]*(L-1) -> LINEAR -> SIGMOID.
            AL, caches = L_model_forward(X, parameters)

            # Compute cost.
            cost = compute_cost(AL, Y)

            # Backward propagation.
            grads = L_model_backward(AL, Y, caches)

            # Update parameters. 
            parameters = update_parameters(parameters, grads, learning_rate)

            # Print the cost every 100 training example
            if print_cost and i % 100 == 0:
                print ("Cost after iteration % i: % f" %(i, cost))
            if print_cost and i % 100 == 0:
                costs.append(cost)

        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per hundreds)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        return parameters
    
    def predict(parameters, path_image):
 
        my_image = path_image
        image = np.array(ndimage.imread(my_image, flatten = False))
        my_image = scipy.misc.imresize(image,
                      size =(num_px, num_px)).reshape((
                              num_px * num_px * 3, 1))

        my_image = my_image / 255.
        output, cache = L_model_forward(my_image, parameters)
        output = np.squeeze(output)
        prediction = round(output)
        if(prediction == 1):
            label = "Cat picture"
        else:
            label = "Non-Cat picture"   # If the model is trained to recognize a cat image.
        print ("y = " + str(prediction) + ", your L-layer model predicts a \"" + label)
        
    

In [3]:
class Linear_Layer(Layer):
    def forward(A,W,b):
        return Layer.linear_forward(A, W, b)
    def backward(dz,cache):
        
        return linear_backward(dZ, cache)
class Sigmoid(Layer):
    def __init(self):
        self.activation_func="Sigmoid"
    def forward(self, X, parameters):
        return Layer.L_model_forward(X, parameters,self.activation_func)
    def backward(self, dz, cache):
        
        return linear_backward(dZ, cache,self.activation_func)

    
class Hyperbolic_Tangent_Function(Layer):
    def __init__(self):
        self.activation_func="Hyperbolic_Tan"

    def forward(self, X, parameters):
        return Layer.L_model_forward(X, parameters,self.activation_func)
    def backward(self, dz, cache):
        
        return linear_backward(dZ, cache,self.activation_func)
    
class Softmax(Layer):
    def __init__(self):
        self.activation_func="Softmax"

    def forward(self, X, parameters):
        return Layer.L_model_forward(X, parameters,self.activation_func)
    def backward(self, dz, cache):
        
        return linear_backward(dZ, cache,self.activation_func)
class Cross_Entropy_Loss(Layer):
    def __init__(self):
        self.activation_func="Cross_Entropy_Loss"

    def forward(self, X, parameters):
        return Layer.L_model_forward(X, parameters,self.activation_func)
    def backward(self, dz, cache):
        
        return linear_backward(dZ, cache,self.activation_func)
